In [4]:
import pandas as pd
from sqlalchemy import create_engine
# price_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_price_yf.db')
# ticker_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
# ticker_df = pd.read_sql('TOTAL', con=ticker_engine)
# ticker_list = ticker_df.Symbol.to_list()

In [8]:
import tushare as ts
pro = ts.pro_api(token='28ce867873a056747209a2ef0ff53d98ed850903fd954417ac69553b')
price_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_seven_year_price.db')
#获取富时中国50指数
df = pro.index_global(ts_code='SPX', start_date='20000201', end_date='20220913')
df.trade_date = df.trade_date.apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])
# df.to_csv('SPX.csv',index=None)
df.rename(columns={
    'close':'Close', 
    'open':'Open',
    'high':'High',
    'low':'Low',
    'ts_code':'ticker',
    'trade_date':'datetime',
    'vol':'Volume'}, inplace=True)
df = df[['Open','Close','High','Low','Volume','datetime']]
df = df[::-1]

,Open,Close,High,Low,Volume,datetime
0,4037.12,3932.69,4037.12,3921.28,2642497.0,2022-09-13
1,4083.67,4110.41,4119.28,4083.67,2147245.0,2022-09-12
2,4022.94,4067.36,4076.81,4022.94,2097275.0,2022-09-09
3,3959.94,4006.18,4010.50,3944.81,2290058.0,2022-09-08
4,3909.43,3979.87,3987.89,3906.03,2209800.0,2022-09-07
...,...,...,...,...,...,...
3995,1350.64,1353.42,1354.23,1348.60,NaN,2006-10-10
3996,1349.58,1350.66,1352.69,1346.55,NaN,2006-10-09
3997,1352.42,1349.58,1353.22,1344.21,NaN,2006-10-06
3998,1349.84,1353.22,1353.79,1347.75,NaN,2006-10-05


In [14]:
df.index = range(len(df))
df.dropna(inplace=True)
df.to_sql('SPX',con=price_engine,if_exists='replace',index=None)

2802

In [16]:
test = pd.read_sql('SPX',con=price_engine)
test

,Open,Close,High,Low,Volume,datetime
0,1352.39,1343.80,1352.39,1333.71,604565.94,2011-07-08
1,1343.80,1319.49,1343.31,1316.42,634678.00,2011-07-11
2,1319.48,1313.64,1327.17,1313.33,727709.00,2011-07-12
3,1314.45,1317.72,1331.48,1314.45,665194.07,2011-07-13
4,1317.70,1308.87,1326.88,1306.51,720240.94,2011-07-14
...,...,...,...,...,...,...
2797,3909.43,3979.87,3987.89,3906.03,2209800.00,2022-09-07
2798,3959.94,4006.18,4010.50,3944.81,2290058.00,2022-09-08
2799,4022.94,4067.36,4076.81,4022.94,2097275.00,2022-09-09
2800,4083.67,4110.41,4119.28,4083.67,2147245.00,2022-09-12


In [32]:
downtrend_list =[['2022-01-03', '2022-06-21'],
                ['2020-02-20', '2022-03-23'],
                ['2018-10-03', '2018-12-26']]
start_date = downtrend_list[1][0]
end_date = downtrend_list[1][1]
start_index = df[df.datetime == start_date].index.values[0]
end_index = df[df.datetime == end_date].index.values[0]
last_count = abs(start_index - end_index)
last_count

515

In [33]:
ticker = 'AAPL'
ticker = 'BLAT'
ticker = 'VIEW'
ticker = 'SPX'
start_date = '2022-01-03'
start_date = '2020-02-20'
res = pd.DataFrame()
ticker_df = pd.read_sql('{}'.format(ticker),con=price_engine)
# 初始价格
start_price = ticker_df[ticker_df.datetime == start_date].Close.values[0]
# 初始价格时的index
tstart_index = ticker_df[ticker_df.datetime == start_date].index.values[0]
# 下降中的最低价
lowest_price = min(ticker_df[tstart_index:tstart_index+int(last_count*1.33)].Low)
# 最低价时的index
lowest_index = ticker_df[ticker_df.Low == lowest_price].index.values[0]
# 下降率
decline_rate = round((start_price - lowest_price) / start_price, 3)
# 下降时间
tlast_count = abs(lowest_index - tstart_index)
# 更新df的范围，起始点为最低价
ticker_df = ticker_df[lowest_index:]

# 获取到达原价75%时的index和价格
if ticker_df[ticker_df.Close >= start_price * 0.75].empty != True:
    rev_h_index = ticker_df[ticker_df.Close >= start_price * 0.75].index.values[0]
    rev_h_price = ticker_df[ticker_df.Close >= start_price * 0.75].Close.values[0]
else:
    rev_h_index = 9999
    rev_h_price = 9999
# 获取到达最低价125%时的index
if ticker_df[ticker_df.Close >= lowest_price * 1.25].empty != True:
    rev_l_index = ticker_df[ticker_df.Close >= lowest_price * 1.25].index.values[0]
    rev_l_price = ticker_df[ticker_df.Close >= lowest_price * 1.25].Close.values[0]
else: 
    rev_l_index = 9999
    rev_l_price = 9999

# 进场点（用于计算收益，取小的index）
# get enter_point和enter_prices
if rev_h_index == rev_l_index == 9999:
    print('can not continue')
    # return pd.DataFrame()
elif rev_h_index == 9999:
    rev_h_count = 9999
    rev_l_count = abs(rev_l_index - lowest_index)
    enter_point = rev_l_count
    enter_price = rev_l_price
elif rev_l_index == 9999:
    rev_l_count = 9999
    rev_h_count = abs(rev_h_index - lowest_index)
    enter_point = rev_h_count
    enter_price = rev_h_price
else:
    # 分别计算到达的时间
    rev_h_count = abs(rev_h_index - lowest_index)
    rev_l_count = abs(rev_l_index - lowest_index)
    if rev_l_count < rev_h_count:
        enter_point = rev_l_count
        enter_price = rev_l_price
    else:
        enter_point = rev_h_count
        enter_price = rev_h_price

# 更新df范围，起始点变为进场点
ticker_df = ticker_df[enter_point:]
max_30 = max(ticker_df[:int(30/7*5)].Close)
max_60 = max(ticker_df[:int(60/7*5)].Close)
max_90 = max(ticker_df[:int(90/7*5)].Close)
max_180 = max(ticker_df[:int(180/7*5)].Close)
profit_30 = (max_30 - enter_price) / enter_price
profit_60 = (max_60 - enter_price) / enter_price
profit_90 = (max_90 - enter_price) / enter_price
profit_180 = (max_180 - enter_price) / enter_price
temp = pd.DataFrame([[ticker,decline_rate,tlast_count,rev_h_count, rev_l_count, profit_30,profit_60,profit_90,profit_180]],columns=['ticker','dec_rate','dec_time','rec_h_time','rec_l_time','pft_30','pft_60','pft_90','pft_180'])
res = pd.concat([res,temp],ignore_index=True)
res

,ticker,dec_rate,dec_time,rec_h_time,rec_l_time,pft_30,pft_60,pft_90,pft_180
0,SPX,0.35,22,3,12,0.09296,0.137525,0.229013,0.3615


In [35]:
temp = pd.concat([temp,res],ignore_index=True)
temp

,ticker,dec_rate,dec_time,rec_h_time,rec_l_time,pft_30,pft_60,pft_90,pft_180
0,SPX,0.35,22,3,12,0.09296,0.137525,0.229013,0.3615
1,SPX,0.35,22,3,12,0.09296,0.137525,0.229013,0.3615


In [8]:
test = pd.DataFrame()
res = pd.concat([res, test],ignore_index=True)
res

,ticker,dec_rate,dec_time,rec_h_time,rec_l_time,pft_30,pft_60,pft_90,pft_180
0,AAPL,0.388,62,22,45,0.009588,0.123493,0.215696,0.223103


In [104]:
temp

,ticker,dec_rate,dec_time,rec_l_time,rec_h_time,pft_30,pft_60,pft_90,pft_180


In [101]:
decline_rate,tlast_count

(0.388, 62)

In [102]:
rev_h_count,rev_l_count

(84, 107)

In [100]:
profit_30,profit_60,profit_90,profit_180,

(0.16596439508661723,
 0.16596439508661723,
 0.21317253233178732,
 0.32449102752452325)

ValueError: max() arg is an empty sequence

In [69]:
profit_30,profit_60,profit_90,profit_180

(-1376.3594973827155,
 -1376.3594973827155,
 -1376.3594973827155,
 -1376.3594973827155)

In [63]:
recover_l25_index, rev_h_count

(10325, 256)

In [65]:
rev_l_count, rev_h_count

(27, 256)

In [28]:
aapl[aapl.datetime == '2018-12-26']

,open,hgih,low,close,Adj Close,volume,datetime
9591,37.075001,39.307499,36.68,39.2925,38.039845,234330000,2018-12-26
